In [1]:
import geopandas as gpd
import fiona
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import configparser
import momepy
import libpysal
import osmnx
import pandas
from bokeh.io import output_notebook
from bokeh.plotting import show
from clustergram import Clustergram
from shapely.geometry import Point
local_crs = "EPSG:2039"
output_notebook()

c:\Users\david\Desktop\DSSG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BokehJS ...

Load all the gdb files from michael folders

In [2]:

config = configparser.ConfigParser()
config_file = 'config.ini'
# check if file exists
if not os.path.isfile(config_file):
    # create a simple config file
    config['Paths'] = {'root_folder': '/path/to/your/gdb/folder'}
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

config.read('config.ini')

root_folder = config['Paths']['root_folder']  
print(root_folder)
root_folder = os.path.join(root_folder, "DSSG/Michaels_Data/All_Layers")
print(root_folder)
# List all .gdb directories in the folder
gdb_files = []
for dirpath, dirnames, filenames in os.walk(root_folder):
    # Check if the folder contains a "commondata" folder
    if "commondata" in dirnames:
        commondata_folder = os.path.join(dirpath, "commondata")
        commondata_folder = os.path.normpath(commondata_folder)

        # List all .gdb files in the "commondata" folder
        for f in os.listdir(commondata_folder):
            if f.endswith('.gdb'):
                gdb_files.append(os.path.join(commondata_folder, f))

# Print the collected .gdb files
for gdb in gdb_files:
    print(gdb)
print("GDB Files:", gdb_files)

C:\\Users\\david\\Desktop
C:\\Users\\david\\Desktop\DSSG/Michaels_Data/All_Layers
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\commondata\myproject16.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\commondata\scratch.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\חלקות\commondata\myproject16.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\חלקות\commondata\scratch.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\מרקמים\commondata\myproject16.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\מרקמים\commondata\scratch.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\קונטור בניינים\commondata\jps_reka.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\קונטור בניינים\commondata\scratch.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\רגישות\commondata\myproject16.gdb
C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\קונטור בניינים\commondata\scratch.gdb


Calculating metrics over buildings:

In [3]:
# load municipal buildings
gdb_file = gdb_files[6]  # Modify if you want to choose a different .gdb
print(gdb_file)
# List the layers in the selected .gdb
layers = fiona.listlayers(gdb_file)
print("Layers in the selected GDB:", layers)

# Choose a specific layer within the .gdb
textures_layer = layers[0]  # Modify if you want to choose a different layer

# Load the specific layer
gdf = gpd.read_file(gdb_file, layer=textures_layer)
buildings = gdf[["geometry"]].to_crs(local_crs)


C:\Users\david\Desktop\DSSG\Michaels_Data\All_Layers\מרקמים\קונטור בניינים\commondata\jps_reka.gdb
Layers in the selected GDB: ['buildings']


Calculate metrics over streets

In [4]:
place = "Jerusalem, Israel"
local_crs = "EPSG:2039"

# Geocode using Nominatim
# latitude = 31.7683
# longitude = 35.2137
# point = gpd.GeoDataFrame(geometry=[Point(longitude, latitude)], crs="EPSG:4326")

#Load open street buildings
osm_buildings = osmnx.features_from_place(place, tags={"building": True})
osm_buildings = osm_buildings[osm_buildings.geom_type == "Polygon"].reset_index(drop=True)
osm_buildings = osm_buildings[["geometry"]].to_crs(local_crs)
osm_buildings.head()
osm_graph = osmnx.graph_from_place(place, network_type="drive")
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
streets = osmnx.graph_to_gdfs(
    osmnx.convert.to_undirected(osm_graph),
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True,
).reset_index(drop=True)
print(streets.columns)
streets_geometry = streets['geometry']
streets['orientation'] = momepy.orientation(streets_geometry)
streets['courtyard_area'] = momepy.courtyard_area(streets_geometry)
streets['longest_axis_length'] = momepy.longest_axis_length(streets_geometry)
# streets['perimeter_wall'] = momepy.perimeter_wall(streets_geometry)
streets['centroid_corner_distance'] = momepy.centroid_corner_distance(streets_geometry)
streets['circular_compactness'] = momepy.circular_compactness(streets_geometry)
streets['compactness_weighted_axis'] = momepy.compactness_weighted_axis(streets_geometry)
streets['convexity'] = momepy.convexity(streets_geometry)
streets['corners'] = momepy.corners(streets_geometry)
streets['courtyard_index'] = momepy.courtyard_index(streets_geometry)
streets['elongation'] = momepy.elongation(streets_geometry)
streets['equivalent_rectangular_index'] = momepy.equivalent_rectangular_index(streets_geometry)
streets['facade_ratio'] = momepy.facade_ratio(streets_geometry)
streets['fractal_dimension'] = momepy.fractal_dimension(streets_geometry)
streets['linearity'] = momepy.linearity(streets_geometry)
streets['rectangularity'] = momepy.rectangularity(streets_geometry)
streets['shape_index'] = momepy.shape_index(streets_geometry)
streets['square_compactness'] = momepy.square_compactness(streets_geometry)
streets['squareness'] = momepy.elongation(streets_geometry)
streets['alignment'] = momepy.alignment(streets['orientation'], osm_graph)
streets['cell_alignment'] = momepy.cell_alignment(streets['orientation'], osm_graph)
streets['neighbor_distance'] = momepy.neighbor_distance(streets_geometry, osm_graph)
streets['neighbors'] = momepy.neighbors(streets_geometry, osm_graph)
streets['betweenness_centrality'] = momepy.betweenness_centrality(osm_graph)



streets = momepy.remove_false_nodes(streets)
streets["length"] = streets.length
streets["linearity"] = momepy.linearity(streets)

Index(['osmid', 'oneway', 'name', 'highway', 'reversed', 'length', 'from',
       'to', 'geometry', 'lanes', 'ref', 'maxspeed', 'tunnel', 'access',
       'junction', 'bridge', 'width', 'service'],
      dtype='object')


KeyError: "None of [Index(['geometry'], dtype='object')] are in the [index]"

common metrics:

In [ ]:
#street_profile